In [25]:
import numpy as np
import tensorflow as tf
import datetime
import contextlib
import os

tf.logging.set_verbosity(tf.logging.INFO)

TRAINING_PROPORTION = 0.8

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
@contextlib.contextmanager
def record_time():
    start_time = datetime.datetime.now()
    yield
    end_time = datetime.datetime.now()
    print("Time taken: {result}".format(result=end_time - start_time))

In [4]:
batch_1 = unpickle('cifar-10-batches-py/data_batch_1')
training = batch_1

In [15]:
batch_1[b'labels']

[6,
 9,
 9,
 4,
 1,
 1,
 2,
 7,
 8,
 3,
 4,
 7,
 7,
 2,
 9,
 9,
 9,
 3,
 2,
 6,
 4,
 3,
 6,
 6,
 2,
 6,
 3,
 5,
 4,
 0,
 0,
 9,
 1,
 3,
 4,
 0,
 3,
 7,
 3,
 3,
 5,
 2,
 2,
 7,
 1,
 1,
 1,
 2,
 2,
 0,
 9,
 5,
 7,
 9,
 2,
 2,
 5,
 2,
 4,
 3,
 1,
 1,
 8,
 2,
 1,
 1,
 4,
 9,
 7,
 8,
 5,
 9,
 6,
 7,
 3,
 1,
 9,
 0,
 3,
 1,
 3,
 5,
 4,
 5,
 7,
 7,
 4,
 7,
 9,
 4,
 2,
 3,
 8,
 0,
 1,
 6,
 1,
 1,
 4,
 1,
 8,
 3,
 9,
 6,
 6,
 1,
 8,
 5,
 2,
 9,
 9,
 8,
 1,
 7,
 7,
 0,
 0,
 6,
 9,
 1,
 2,
 2,
 9,
 2,
 6,
 6,
 1,
 9,
 5,
 0,
 4,
 7,
 6,
 7,
 1,
 8,
 1,
 1,
 2,
 8,
 1,
 3,
 3,
 6,
 2,
 4,
 9,
 9,
 5,
 4,
 3,
 6,
 7,
 4,
 6,
 8,
 5,
 5,
 4,
 3,
 1,
 8,
 4,
 7,
 6,
 0,
 9,
 5,
 1,
 3,
 8,
 2,
 7,
 5,
 3,
 4,
 1,
 5,
 7,
 0,
 4,
 7,
 5,
 5,
 1,
 0,
 9,
 6,
 9,
 0,
 8,
 7,
 8,
 8,
 2,
 5,
 2,
 3,
 5,
 0,
 6,
 1,
 9,
 3,
 6,
 9,
 1,
 3,
 9,
 6,
 6,
 7,
 1,
 0,
 9,
 5,
 8,
 5,
 2,
 9,
 0,
 8,
 8,
 0,
 6,
 9,
 1,
 1,
 6,
 3,
 7,
 6,
 6,
 0,
 6,
 6,
 1,
 7,
 1,
 5,
 8,
 3,
 6,
 6,
 8,
 6,
 8,
 4,
 6,
 6,


In [5]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session()

# Runs the op.
options = tf.RunOptions(output_partition_graphs=True)
metadata = tf.RunMetadata()
c_val = sess.run(c, options=options, run_metadata=metadata)

print(metadata.partition_graphs)

[node {
  name: "MatMul/_0"
  op: "_Send"
  input: "MatMul/_0__cf__0"
  device: "/job:localhost/replica:0/task:0/gpu:0"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "client_terminated"
    value {
      b: false
    }
  }
  attr {
    key: "recv_device"
    value {
      s: "/job:localhost/replica:0/task:0/cpu:0"
    }
  }
  attr {
    key: "send_device"
    value {
      s: "/job:localhost/replica:0/task:0/gpu:0"
    }
  }
  attr {
    key: "send_device_incarnation"
    value {
      i: 1
    }
  }
  attr {
    key: "tensor_name"
    value {
      s: "edge_8_MatMul"
    }
  }
}
node {
  name: "MatMul/_0__cf__0"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/gpu:0"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
   

In [16]:
def cnn_model_fn(features, labels, mode):
    data = features['x']
    
    input_layer = tf.reshape(data, [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2)
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=2)

    # Dense Layer
    flat = tf.reshape(pool2, [-1, 64 * 8 * 8])
    dense = tf.layers.dense(inputs=flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"]),
        "auc": tf.metrics.auc(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

cifar_cfr = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir='convnet')

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

tf.reset_default_graph()
train_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(batch_1[b'data'][:int(TRAINING_PROPORTION * len(batch_1[b'data']))], dtype=np.float16)},
    y = np.array(batch_1[b'labels'][:int(TRAINING_PROPORTION * len(batch_1[b'labels']))]),
    num_epochs = None,
    shuffle = False)

with record_time():
    cifar_cfr.train(train_fn, steps=1000)
    
test_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(batch_1[b'data'][int(TRAINING_PROPORTION * len(batch_1[b'data'])):], dtype=np.float16)},
    y = np.array(batch_1[b'labels'][int(TRAINING_PROPORTION * len(batch_1[b'labels'])):]),
    num_epochs = 1,
    shuffle = False)
with record_time():
    evaluation_results = cifar_cfr.evaluate(test_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'convnet', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into convnet/model.ckpt.
INFO:tensorflow:loss = 78.688, step = 1
INFO:tensorflow:global_step/sec: 70.962
INFO:tensorflow:loss = 2.3965, step = 101 (1.410 sec)
INFO:tensorflow:global_step/sec: 72.5033
INFO:tensorflow:loss = 2.3594, step = 201 (1.379 sec)
INFO:tensorflow:global_step/sec: 72.1539
INFO:tensorflow:loss = 2.3359, step = 301 (1.386 sec)
INFO:tensorflow:global_step/sec: 72.8788
INFO:tensorflow:loss = 2.3203, step = 401 (1.372 sec)
INFO:tensorflow:global_step/sec: 72.8771
INFO:tensorflow:loss = 2.2949, step = 501 (1.372 sec)
INFO:tensorflow:global_step/sec: 72.9487
INFO

In [37]:
def alexnet_model_fn(features, labels, mode):
    data = features['x']
    
    input_layer = tf.reshape(data, [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=96,
        kernel_size=11,
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2)
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=256,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=2)
    
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=384,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides=2)
    
    conv4 = tf.layers.conv2d(
        inputs=pool3,
        filters=384,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=2, strides=2)
    
    conv5 = tf.layers.conv2d(
        inputs=pool4,
        filters=256,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=2, strides=2)

    # Dense Layer
    flat = tf.reshape(pool5, [-1, 256 * 1 * 1])
    dense1 = tf.layers.dense(inputs=flat, units=2048, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(
        inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense2 = tf.layers.dense(inputs=dropout1, units=2048, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(
        inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout2, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"]),
        "auc": tf.metrics.auc(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

directory = 'alexnet'
try:
    os.removedirs(directory)
except OSError:
    pass
alex_cfr = tf.estimator.Estimator(model_fn=alexnet_model_fn, model_dir=directory)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

tf.reset_default_graph()
train_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(batch_1[b'data'][:int(TRAINING_PROPORTION * len(batch_1[b'data']))], dtype=np.float16)},
    y = np.array(batch_1[b'labels'][:int(TRAINING_PROPORTION * len(batch_1[b'labels']))]),
    num_epochs = None,
    shuffle = False)

with record_time():
    alex_cfr.train(train_fn, steps=500)
    
test_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(batch_1[b'data'][int(TRAINING_PROPORTION * len(batch_1[b'data'])):], dtype=np.float16)},
    y = np.array(batch_1[b'labels'][int(TRAINING_PROPORTION * len(batch_1[b'labels'])):]),
    num_epochs = 1,
    shuffle = False)
with record_time():
    evaluation_results = alex_cfr.evaluate(test_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'alexnet', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'report_uninitialized_variables_1/boolean_mask/Gather:0' shape=(?,) dtype=string>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/home/will/anaconda2/envs/tensor/lib/python3.6/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/home/will/anaconda2/envs/tensor/lib/python3.6/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/home/will/anaconda2/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <

In [53]:
def zfnet_model_fn(features, labels, mode):
    data = features['x']
    
    input_layer = tf.reshape(data, [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=96,
        kernel_size=7,
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=3, strides=2)
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=256,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=3, strides=1)
    
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=384,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=3, strides=1)
    
    conv4 = tf.layers.conv2d(
        inputs=pool3,
        filters=384,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=3, strides=1)
    
    conv5 = tf.layers.conv2d(
        inputs=pool4,
        filters=256,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=3, strides=2)

    # Dense Layer
    flat = tf.reshape(pool5, [-1, 256 * 4 * 4])
    dense1 = tf.layers.dense(inputs=flat, units=2048, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(
        inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense2 = tf.layers.dense(inputs=dropout1, units=2048, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(
        inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout2, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"]),
        "auc": tf.metrics.auc(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

directory = 'zfnet'
try:
    os.removedirs(directory)
except OSError:
    pass
zf_cfr = tf.estimator.Estimator(model_fn=zfnet_model_fn, model_dir=directory)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

tf.reset_default_graph()
train_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(batch_1[b'data'][:int(TRAINING_PROPORTION * len(batch_1[b'data']))], dtype=np.float16)},
    y = np.array(batch_1[b'labels'][:int(TRAINING_PROPORTION * len(batch_1[b'labels']))]),
    num_epochs = None,
    shuffle = False)

with record_time():
    zf_cfr.train(train_fn, steps=500)
    
test_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(batch_1[b'data'][int(TRAINING_PROPORTION * len(batch_1[b'data'])):], dtype=np.float16)},
    y = np.array(batch_1[b'labels'][int(TRAINING_PROPORTION * len(batch_1[b'labels'])):]),
    num_epochs = 1,
    shuffle = False)
with record_time():
    evaluation_results = zf_cfr.evaluate(test_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'zfnet', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from zfnet/model.ckpt-501
INFO:tensorflow:Saving checkpoints for 502 into zfnet/model.ckpt.
INFO:tensorflow:loss = 1.8408, step = 502
INFO:tensorflow:global_step/sec: 7.48209
INFO:tensorflow:loss = 1.874, step = 602 (13.366 sec)
INFO:tensorflow:global_step/sec: 7.54179
INFO:tensorflow:loss = 1.8809, step = 702 (13.260 sec)
INFO:tensorflow:global_step/sec: 7.53514
INFO:tensorflow:loss = 1.9443, step = 802 (13.271 sec)
INFO:tensorflow:global_step/sec: 7.53118
INFO:tensorflow:loss = 1.8496, step = 902 (13.278 sec)
INFO:tensorflow:Saving checkpoints for 1001 into zfnet/model.ckpt.
INFO:t

In [58]:
def vggnet_model_fn(features, labels, mode):
    data = features['x']
    
    input_layer = tf.reshape(data, [-1, 32, 32, 3])
    
    conv1a = tf.layers.conv2d(
        inputs=input_layer,
        filters=64,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    conv1 = tf.layers.conv2d(
        inputs=conv1a,
        filters=64,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2)
    
    conv2a = tf.layers.conv2d(
        inputs=pool1,
        filters=128,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    conv2 = tf.layers.conv2d(
        inputs=conv2a,
        filters=128,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=2)
    
    conv3a = tf.layers.conv2d(
        inputs=pool2,
        filters=256,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    conv3b = tf.layers.conv2d(
        inputs=conv3a,
        filters=256,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    conv3 = tf.layers.conv2d(
        inputs=conv3b,
        filters=256,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides=2)
    
    conv4a = tf.layers.conv2d(
        inputs=pool3,
        filters=512,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    conv4b = tf.layers.conv2d(
        inputs=conv4a,
        filters=512,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    conv4 = tf.layers.conv2d(
        inputs=conv4b,
        filters=512,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=2, strides=2)
    
    conv5a = tf.layers.conv2d(
        inputs=pool4,
        filters=512,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    conv5b = tf.layers.conv2d(
        inputs=conv5a,
        filters=512,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    conv5 = tf.layers.conv2d(
        inputs=conv5b,
        filters=512,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=2, strides=2)

    # Dense Layer
    flat = tf.reshape(pool5, [-1, 512 * 1 * 1])
    dense1 = tf.layers.dense(inputs=flat, units=4096, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(
        inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense2 = tf.layers.dense(inputs=dropout1, units=4096, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(
        inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense3 = tf.layers.dense(inputs=dropout2, units=1000, activation=tf.nn.relu)
    dropout3 = tf.layers.dropout(
        inputs=dense3, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout3, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"]),
        "auc": tf.metrics.auc(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

directory = 'vggnet'
try:
    os.removedirs(directory)
except OSError:
    pass
vgg_cfr = tf.estimator.Estimator(model_fn=vggnet_model_fn, model_dir=directory)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

tf.reset_default_graph()
train_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(batch_1[b'data'][:int(TRAINING_PROPORTION * len(batch_1[b'data']))], dtype=np.float16)},
    y = np.array(batch_1[b'labels'][:int(TRAINING_PROPORTION * len(batch_1[b'labels']))]),
    num_epochs = None,
    shuffle = False)

with record_time():
    vgg_cfr.train(train_fn, steps=500)
    
test_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(batch_1[b'data'][int(TRAINING_PROPORTION * len(batch_1[b'data'])):], dtype=np.float16)},
    y = np.array(batch_1[b'labels'][int(TRAINING_PROPORTION * len(batch_1[b'labels'])):]),
    num_epochs = 1,
    shuffle = False)
with record_time():
    evaluation_results = vgg_cfr.evaluate(test_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'vggnet', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'report_uninitialized_variables_1/boolean_mask/Gather:0' shape=(?,) dtype=string>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/home/will/anaconda2/envs/tensor/lib/python3.6/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/home/will/anaconda2/envs/tensor/lib/python3.6/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/home/will/anaconda2/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <m